In [2]:
import dotenv
import pymongo
import os

dotenv.load_dotenv()

MONGO_URL = os.getenv('MONGO_URI')

client = pymongo.MongoClient(MONGO_URL)
db = client['shopify-inventory']

In [9]:
def searchShopifyInventory(search_term, db):
    mtgSinglesCollection = db['mtgSingles']
    # case insensitive and punctuation insensitive using full text search on index "title"
    result = list(mtgSinglesCollection.find({"$text": {"$search": search_term}}))
    # drop the _id field from the result
    for item in result:
        item.pop('_id')
    return result



In [10]:
search_term = "fblthp the lost"
results = searchShopifyInventory(search_term, db)
for result in results:
    print(f"Found {result['title']} in {result['set']} with {result['website']} for ${result['price']}")

Found Fblthp, the Lost in War of the Spark with gamezilla for $1.89
Found Fblthp, the Lost in War of the Spark with gamezilla for $1.99
Found Fblthp, the Lost in War of the Spark Prerelease Promos with gamezilla for $2.10
Found Fblthp, the Lost in Mystery Booster with gamezilla for $1.89
Found Borborygmos and Fblthp in March of the Machine with gamezilla for $3.79
Found Borborygmos and Fblthp in March of the Machine with gamezilla for $3.99
Found Fblthp, the Lost in War of the Spark with gameknight for $0.90
Found Fblthp, the Lost in War of the Spark with gameknight for $1.00
Found Fblthp, the Lost in War of the Spark Prerelease Promos with gameknight for $2.20
Found Fblthp, the Lost in Mystery Booster with gameknight for $1.00
Found Borborygmos and Fblthp in March of the Machine with gameknight for $2.00
Found Borborygmos and Fblthp in March of the Machine with gameknight for $2.00
Found Borborygmos and Fblthp in March of the Machine Prerelease Promos with gameknight for $3.70
Found F

In [14]:
# print the type of the result    
# print type of each key 

# _id
# 64ce5c909e03410b422ffc9b
# title
# "Stormwatch Eagle"
# handle
# "stormwatch-eagle-the-list"
# website
# "hairyt"
# image
# "https://cdn.shopify.com/s/files/1/0266/9513/9533/products/4d4b8244-2c7…"
# link
# "https://hairyt.com/products/stormwatch-eagle-the-list"
# set
# "The List"
# variantTitle
# "NM"
# foil
# false
# price
# "0.30"

print(type(result['title']))
print(type(result['handle']))
print(type(result['website']))
print(type(result['image']))
print(type(result['link']))
print(type(result['set']))
print(type(result['variantTitle']))
print(type(result['foil']))
print(type(result['price']))
      



<class 'dict'>
dict_keys(['title', 'handle', 'website', 'image', 'link', 'set', 'variantTitle', 'foil', 'price'])
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'bool'>
<class 'str'>


In [22]:
from pymongo import UpdateOne

# find any documents with a price that's a string
mtgSinglesCollection = db['mtgSingles']
priceStringCards = mtgSinglesCollection.find({'price': {'$type': 'string'}})

operations = []

for card in priceStringCards:
    try:
        new_price = float(card['price'])
        operations.append(UpdateOne({'_id': card['_id']}, {'$set': {'price': new_price}}))
    except ValueError:
        print(f"Cannot convert price: {card['price']} to float")

if operations:
    mtgSinglesCollection.bulk_write(operations)
